# MODELOS PREDICTIVOS CON PYTHON

## Introducción

 Realizaremos analisis de prediccion sobre una variable de valor, el objetivo general es ajustar un modelo de variables significativas que nos ayuden a predecir en un porcentaje de certeza alto el valor de la variable de valor.

**Nota**: para la realizacion de este desafio utilizaremos las siguientes librerias **numpy**, **pandas**, **seaborn**, **matplotlib**, **openpyxl**, **sklearn**.

***

## Cargamos las librerias a utilizar

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

## Se carga el data set

In [ ]:
path = './datos/fifa_datos_limpios.csv'
df = pd.read_csv(path)
len(df)

In [ ]:
import io
from google.colab import files

uploaded = files.upload()

In [ ]:
df = pd.read_csv(io.BytesIO(uploaded['fifa_datos_limpios.csv']))
print(df)

## Vemos graficamente las tablas

In [ ]:
# print(df.info())
df.head()

In [ ]:
print(df.dtypes)

Se analiza correlacion de las columnas Potential, Overall y International Reputation con Value-NUM

In [ ]:
# df[['Potential', 'Overall','Value-NUM']].corr()
df.corr().sort_values(by='Value-NUM')['Value-NUM']

In [ ]:
sns.regplot(x="Potential", y="Value-NUM", data=df)
plt.ylim(0,)

In [ ]:
sns.regplot(x="Overall", y="Value-NUM", data=df)
plt.ylim(0,)

In [ ]:
sns.regplot(x="International Reputation", y="Value-NUM", data=df)
plt.ylim(0,)

In [ ]:
visualizar = df[['Potential','Overall','International Reputation']]
visualizar.hist()
plt.show()

Se puede observar que se presenta una tendendia lineal con la variable Potential y Overall pero no se logra apreciar una relacion lineal con la variable International Reputation, aparentar tener un comportamiento discreto. Tambien se puede ver en el histograma un comportamiento normal en las variables Potencial y Overall

## Analisis de variables categóricas

In [ ]:
sns.boxplot(x="International Reputation", y="Value-NUM", data=df)

Se puede observar que la distrubion de la variable Value-NUM entre las diferentes categorias de la variables International Reputation es diferente, por lo que podria ser una variable predictora de la variables Value-NUM

## Evaluacion de Correlacion de Pearson y valor P

In [ ]:
from scipy import stats
coeficiente_pearson, valor_p = stats.pearsonr(df['Potential'], df['Value-NUM'])
print("El coeficiente de la Correlación de Pearson es", coeficiente_pearson, " con un valor de P =", valor_p)

In [ ]:
coeficiente_pearson, valor_p = stats.pearsonr(df['Overall'], df['Value-NUM'])
print("El coeficiente de la Correlación de Pearson es", coeficiente_pearson, " con un valor de P =", valor_p)

Se observa que en ambos casos tenemos un valor P igual a cero, esto indicandonos que tienen una fuerte relacion con la variable Value-NUM

## $\therefore$ Conclusión: Variables importantes

Variables relevantes para predecir el valor de un jugador son las siguientes:

Variables numéricas continuas:
- Potential
- Overall

Variables discretas:
- International Reputation

# Desarrollo del Modelo


In [ ]:
df_var_numericas = df._get_numeric_data()
df_var_numericas = df_var_numericas.drop(['Unnamed: 0'], axis=1)
df_var_numericas.head()

Se dividen los datos en un conjunto de entrenamiento y otro conjunto de pruebas

In [ ]:
# valor a predecir
y_datos = df_var_numericas[['Value-NUM']]
y_datos

In [ ]:
#variables independientes
x_datos=df_var_numericas.drop('Value-NUM',axis=1)
x_datos

In [ ]:
from sklearn.model_selection import train_test_split

x_entrena, x_prueba, y_entrena, y_prueba = train_test_split(x_datos, y_datos, test_size=0.10, random_state=1)

print("número de muestras de prueba:", x_prueba.shape[0])
print("número de muestras de entrenamiento:",x_entrena.shape[0])

Se implementa el modelo lineal

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lreP=LinearRegression()
lreP

Se implementa un modelo lineal utilizando la variable Potential como variable independiente

In [ ]:
lreP.fit(x_entrena[['Potential']], y_entrena)


Una vez terminado el entrenamiento del modelo lineal utilizando la variable Potential como variable independiente se procede a calcular el valor de $R^2$ en los datos de prueba.

In [ ]:
print("R cuadrado datos de prueba: ", lreP.score(x_prueba[['Potential']], y_prueba))
print("R cuadrado datos de entrenamiento: ", lreP.score(x_entrena[['Potential']], y_entrena))

 Se observa que el valor de $R^2$ es similar al usando en los datos de prueba con los datos de entrenamiento.

Porbamos un modelo lineal utilizando la variable independiente Overall 

In [ ]:
lreO=LinearRegression()
lreO

In [ ]:
lreO.fit(x_entrena[['Overall']], y_entrena)

In [ ]:
print("R cuadrado datos de prueba: ", lreO.score(x_prueba[['Overall']], y_prueba))
print("R cuadrado datos de entrenamiento: ", lreO.score(x_entrena[['Overall']], y_entrena))


Se puede concluir que utilizando Overall tiene un poco mejor el R cuadrado pero no es tan significativo 

Se implementa modelo Lineal Múltiple

In [ ]:
lre_multiple=LinearRegression()
lre_multiple

In [ ]:
Z = x_entrena[['Potential', 'Overall']]

In [ ]:
Z

In [ ]:
Z.hist()
plt.show()

In [ ]:
lre_multiple.fit(Z, y_entrena)

In [ ]:
print("R cuadrado datos de entrenamiento: ", lre_multiple.score(Z, y_entrena))
print("R cuadrado datos de prueba: ", lre_multiple.score(x_prueba[['Potential', 'Overall']], y_prueba))

Se observa una mejora en el R cuadrado pero aun sigue siendo menor al 50%. Se procede a utlizar modelos polinomiales


## Modelo polinomial

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
# se crea un modelo de grado 2
pr=PolynomialFeatures(degree=2)
pr

In [ ]:
Z_pr=pr.fit_transform(Z)

In [ ]:
Z_pr

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
Entrada=[('normalizar',StandardScaler()), ('polinomio', PolynomialFeatures(include_bias=False)), ('modelo',LinearRegression())]

In [ ]:
tuberia=Pipeline(Entrada)
tuberia

In [ ]:
Z = Z.astype(float)
tuberia.fit(Z,y_entrena)

In [ ]:
y_tubo=tuberia.predict(Z)
y_tubo[0:4]

Se calcula el R cuadrado para evaluar su precision

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
r_cuadrado = r2_score(y_entrena, y_tubo)
print('El valor de R-cuadrado es: ', r_cuadrado)

Se puede ver una mejora significativa en el modelo, teniendo un R cuadrado mayor al 0.80 en los datos de entrenamiento

In [ ]:
Z_test = x_prueba[['Potential', 'Overall']]
y_tubo_test=tuberia.predict(Z_test)
y_tubo_test[0:4]

In [ ]:
 
print('El valor de R-cuadrado datos test es: ', r2_score(y_prueba, y_tubo_test))

Se observa que el R cuadrado en datos de prueba es mayor a 0,79, lo que nos da un resultado de mejor calidad en comparacion a los modelos lineales que obtienen un R cuadrado inferior al 0,50. Concluyendo que el mejor modelo que se ajusta a los datos es el modelo polinomial

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
R_cruceP = cross_val_score(lreP, x_prueba[['Potential']], y_prueba, cv=4)
R_cruceO = cross_val_score(lreO, x_prueba[['Overall']], y_prueba, cv=4)
R_crucePoli = cross_val_score(tuberia, Z_test, y_prueba, cv=4)
print("Potential. El promedio de cada iteración es ", R_cruceP.mean(), "y la desviación estándar es " , R_cruceP.std())
print("Overall. El promedio de cada iteración es ", R_cruceO.mean(), "y la desviación estándar es " , R_cruceO.std())
print("Overall. El promedio de cada iteración es ", R_crucePoli.mean(), "y la desviación estándar es " , R_crucePoli.std())

Entodas las pruebas, el modelo Polinomial sigue siendo el mejor para predecir el valor de un jugador